# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from config import g_key, weather_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path = '../output_data/output_cities.csv'
city_df = pd.read_csv(path)
city_df

,Unnamed: 0,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,0,ostrovnoy,47,RU,1558378754,80,68.05,39.51,40.55,6.11
1,1,1,mahebourg,75,MU,1558378503,83,-20.41,57.70,71.60,11.41
2,2,2,qaanaaq,0,GL,1558378755,79,77.48,-69.36,14.52,7.14
3,3,3,zhuhai,82,CN,1558378755,82,40.71,112.04,55.40,2.24
4,4,4,cape town,20,ZA,1558378755,62,-33.93,18.42,57.00,9.17
...,...,...,...,...,...,...,...,...,...,...,...
543,543,543,yabrud,0,SY,1558378840,28,33.97,36.66,80.60,5.82
544,544,544,paraiso,5,MX,1558378898,37,24.01,-104.61,77.00,4.97
545,545,545,veraval,99,FR,1558378898,88,49.65,0.71,54.00,16.84
546,546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.89,4.27


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
coords = []
for index,row in city_df.iterrows():
    coords.append((row['Lat'], row['Lng']))
fig = gmaps.figure(map_type='HYBRID')
weight = city_df['Humidity'].astype(float)
heatmap = gmaps.heatmap_layer(coords, weights = weight, dissipating=False, max_intensity=city_df['Humidity'].max(), point_radius=3)
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal = city_df.loc[(city_df['Max Temp']>65) & (city_df['Max Temp']<80) & (city_df['Cloudiness'] <80) & (city_df['Humidity']<60)]
ideal = ideal.drop(columns = 'Unnamed: 0')
ideal

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,25,-18.06,13.84,67.98,7.87
14,14,constitucion,5,MX,1558378757,37,23.99,-104.67,77.00,5.30
30,30,gobabis,0,NaN,1558378567,18,-22.45,18.97,66.51,14.03
38,38,ayan,32,TR,1558378761,38,40.67,33.60,69.64,3.87
50,50,kodiak,20,US,1558378763,48,39.95,-94.76,66.20,29.97
68,68,beja,0,PT,1558378766,41,38.02,-7.87,77.00,10.29
121,121,sitka,73,US,1558378776,33,37.17,-99.65,72.00,28.01
124,124,san felipe,0,CL,1558378777,20,-32.75,-70.73,72.12,5.21
144,144,plainview,1,US,1558378590,31,34.18,-101.71,78.80,19.46
146,146,nardaran,9,AZ,1558378782,56,40.56,50.01,68.00,13.87


In [6]:
gurl = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
hotels = []
locations = []
for index, row in ideal.iterrows():
    lat = str(row['Lat'])
    long =str(row['Lng'])

    params = {'key':g_key,
              'location': f'{lat},{long}',
              'radius':5000,
              'keyword': 'hotel'}
    response = requests.get(gurl, params=params).json()
    try:
        results_hotel = response['results'][0]
        hotels.append(results_hotel['name'])
        locations.append((results_hotel['geometry']['location']['lat'],results_hotel['geometry']['location']['lng']))
    except IndexError:
        hotels.append('None Found')
        locations.append('None Found')


In [8]:
ideal['Hotel Name']=hotels
ideal['Hotel Coords'] = locations
ideal_hotels = ideal.loc[ideal['Hotel Name']!='None Found']


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_hotels.iterrows()]
locations = ideal_hotels['Hotel Coords']

In [10]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))